  # Omidyar Extractives Project 1
## Extract Contract Text (Notebook 7 of 8)
### Hash-based partitition function for segmenting documents prior to clustering

In [1]:
from lib.py_rabin import rabin_partition, example_rabin_partition
import re
import time
import subprocess
import itertools
import cPickle as pickle
import numpy as np
import scipy.spatial.distance as ssd
import scipy.cluster.hierarchy as hcluster
import pandas as pd
import collections as cl
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)
from IPython.core.debugger import Tracer
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [2]:
# remove other characters
char_to_remove = set(['.',',',';',':','-','_','[',']','&','`','@','*','^','|','~',';',':','\'','\"',">","<"]) 
def longstr_clean(longstr):
    longstr = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', longstr) # remove all non-printable characters
    longstr = re.compile(r'<.*?>').sub('', longstr) # strip html markup, e.g. <br>,<div>, etc...
    longstr = longstr.replace("&nbsp",'').replace("&lt;",'').replace("&gt;",'').replace("\\","") 
    longstr = ''.join(i for i in longstr if ord(i)<128 and i not in char_to_remove)
    longstr = longstr.replace('\r','').replace('\n','').replace('\t','')
    longstr = " ".join(longstr.split()).lower() # remove whitespace
    return longstr

In [ ]:
start = time.time()
#df = pd.read_pickle('contract_data/openland_contracts_with_text.pkl') # ~200 
df = pd.read_pickle('contract_data/resource_contracts_with_text.pkl') # ~1500
df.columns = [k.lower().replace(" ","_") for k in df.columns]
print('Time to load documents: %s seconds' % str(time.time()-start) )


# ## uncomment for training
# df = df.iloc[:100]
# ##

df["contract_text_clean"] = df.contract_text.apply(longstr_clean) 
print('Time to clean document text: %s seconds' % str(time.time()-start) )

print('\nTotal documents: %d' % len(df))
df.columns

Time to load documents: 0.765012025833 seconds
Time to clean document text: 146.658810854 seconds

Total documents: 1496


Index([u'ocid', u'category', u'contract_name', u'contract_identifier',
       u'language', u'country_name', u'resource', u'contract_type',
       u'signature_date', u'document_type', u'government_entity',
       u'government_identifier', u'company_name', u'company_address',
       u'jurisdiction_of_incorporation', u'registration_agency',
       u'company_number', u'corporate_grouping', u'participation_share',
       u'open_corporates_link', u'incorporation_date', u'operator',
       u'project_title', u'project_identifier', u'license_name',
       u'license_identifier', u'source_url', u'disclosure_mode',
       u'retrieval_date', u'pdf_url', u'deal_number', u'contract_note',
       u'matrix_page', u'annotation_category', u'annotation_text',
       u'contract_text', u'contract_text_clean'],
      dtype='object')

In [ ]:
def rabin_graph(seriesDocs, paramRabin, flag="char"):
    """
    Cpu and memory optimized optimized graph construction
    
    :param serisDf: pandas series documents    
    :param indexDf: pandas series index (default: range(len(series(df))))
    :param paramRabin: 
    :param flag: similarity metric (default: fraction of text in common)
    
    :return G: NetworkX undirected graph (nodes=docs, edges=doc similarity)   
    """
    pass

fpraw = '_docRabinChunks'
fpsorted = '_docRabinChunks_sorted'
paramRabin = {"avgchunk":16, "minchunk":8, "maxchunk":32, "windowsize":8, "windowslide":1} 
flag="char"

# ----------------------------------
# CREATE GRAPH NODES AND BREAK UP DOCUMENTS INTO RABIN CHUNKS
# 1. identify chunks via 'rabin fingerprint', and w
# 2. break up file marker-2-marker and sort by docFingerprint
start = time.time()
G = nx.Graph() 
with open(fpraw,'w') as fp:
    for k in xrange(len(df.index)):
        a = df.contract_text_clean.iloc[k]   # retrieve document text (must be clean already)
        if len(a) <= 10:  continue
        b = rabin_partition(a,**paramRabin)  # partition document via rabin fingerprint
        c = set(b)

        # add node to graph; 
        # also store document id, unique chunks, character length of unique chunks
        G.add_node(k,
                   ocid = df.ocid.iloc[k],
                   n_chunk = len(c),
                   n_char  = sum([len(i) for i in c]))
        fp.writelines([str(i)+'|'+str(k)+'\n' for i in c])
print('Rabin chunks identified: %s seconds' % str(time.time()-start) )

# ----------------------------------
# in-memory pre-sort for all document chunks for speed up many-many comparison (O(n^2))
cmd = 'sort -k1 -S2G ' + fpraw + ' > ' + fpsorted
subprocess.call(cmd,shell=True)
print('Rabin chunks sorted: %s seconds' % str(time.time()-start) )

# ----------------------------------
# UPDATE GRAPH WITH EDGE WEIGHTS = node similarity
with open(fpsorted,'r') as fp:
    # read line by line
    for ix,line in enumerate(fp):
        chunk,k = line.strip('\n').split('|')
        if ix==0: tmp=[]; prev_chunk = chunk

        # collect docs that share the same chunk (recall: chunks are sorted)
        if len(prev_chunk) == len(chunk) and prev_chunk == chunk:
            tmp.extend([int(k)])

        # complete edge comparison, i.e.common chunks between document pairs    
        else:
            if len(tmp)>1:
                lenchunk = float(len(prev_chunk))
                for (m,n) in itertools.combinations(tmp,2):
                    if not G.has_edge(m,n): 
                        G.add_edge(m, n, n_chunk_matched=0., n_char_matched=0.)
                    G[m][n]['n_chunk_matched'] += 1. 
                    G[m][n]['n_char_matched']  += lenchunk
            tmp=[int(k)]
        prev_chunk = chunk 

    # compute weights for all edges based on jaccard distance = len(shared text) / len(shorter document) 
    for m,n in G.edges_iter():
        if flag == "char":
            G[m][n]["weight"] = G[m][n]["n_char_matched"] / min(G.node[m]['n_char'],G.node[n]['n_char']) 
        elif flag == "chunk":
            G[m][n]["weight"] = G[m][n]["n_chunk_matched"] / min(G.node[m]['n_chunk'],G.node[n]['n_chunk']) 
        else:
            assert True # No such distance metric is available

# clean up tmp files 
cmd = 'rm '+fpraw+' && '+'rm '+fpsorted
subprocess.call(cmd,shell=True)
print('Documents compared for similarity: %s seconds'%str(time.time()-start))

In [ ]:
def identifyIsolates(G):
    """Identify isolated nodes, i.e. nodes not connected to any other"""
    iso = nx.isolates(G)
    return iso if len(iso)>0 else None 
def clipEdges(G,minWeight=0.01):
    """Clip weakly connected edges"""
    if minWeight:
        print('\nClipped edges at Applied min similarity threshold: %f' % minWeight )
        H = G.copy()
        H.remove_edges_from([(u,v) for (u,v,d) in H.edges(data=True) if d['weight'] < minWeight])
    return H if minWeight else G  

# clip weak edges
G = clipEdges(G,minWeight=0.01)

# remove one-off documents (those not connected to any other)
oneoffs  = identifyIsolates(G)        # identify isolated docs
if oneoffs:
    Goneoffs = nx.subgraph(G,oneoffs) # keep info in a separate graph
    G.remove_nodes_from(oneoffs)      # remove oneoffs from the main network
    print("\nNumber of 'oneoff' documents: %d"%len(oneoffs))

# Graph summary
print('\nGraph/network summary:')
print nx.info(G)
print('Average density: %f' % nx.density(G))

### Classic ward hierarchical clustering 

In [ ]:
## classic *ward* hierarchical clustering 
k = 0
N = len(G.nodes())
dMatrix = np.zeros([N*(N-1)/2,1])
for m in G.nodes():
    for n in G.nodes():
        if n>m:
            #if G.has_edge(m,n) and G[m][n]["weight"] > 0.95: print m,n,G[m][n]["weight"]             
            dMatrix[k,0] = 1. - G[m][n]["weight"] if G.has_edge(m,n) else 1.
            k+=1
dMatrix = dMatrix.flatten()
linkage = hcluster.ward(dMatrix)
dendro  = hcluster.dendrogram(linkage,orientation='right')

### Louvain community detection

In [ ]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z=x.copy()
    return z.update(y)

def louvain(G,labelBase=None):
    """
    Hierchical clustering algorithm. Best-in-class winner 2010 
    ref: V.D. Blondel, J.-L. Guillaume, R. Lambiotte, E. Lefebvre. 
         "Fast unfolding of communities in large networks." J. Stat. Mech., 2008: 1008
    
    :param G: NetworkX undirected graph contaning nodes, edges, and edge weights
    :param labelBase: used to convert default 0,1,2... to labelBase.0, labelBase.1, etc... 
    :return D: ordered dictionary containing cluster labels and G.nodes() belonging to each cluster 
    """
    import community as cm
    D=cl.OrderedDict()
    p=cm.best_partition(G)
    for label in set(p.values()):
        newlabel = str(labelBase)+"."+str(int(label)) if labelBase else str(int(label))
        D[newlabel] = sorted([k for k in p.keys() if p[k] == label])
    return D

def recursive_louvain(G,maxLevel=10):
    """
    Recursive wrapper for lovain clustering algorithm
    
    :param G: symmetric networkx graph containg nodes, edges, and edge weights
    :param maxLevel: max recursive depth  
    :return D: ordered dictionary containing cluster labels and G.nodes() belonging to each cluster 
    """
    start = time.time()
    D = cl.OrderedDict()
    for k in xrange(maxLevel):
        prev_level = "Level_%02d"%(k-1)
        next_level = "Level_%02d"%(k)
        d={}    
        # level zero
        if k==0: 
            D[next_level] = louvain(G)
        
        # level 1+
        else: 
            # loop through all clusters at previous level
            for key,val in D[prev_level].iteritems():
                d2 = louvain(G.subgraph(val),key)
                if len(d2.keys())==1: 
                    d = merge_two_dicts(d,{key:val})
                else:
                    try: 
                        d = merge_two_dicts(d,d2)
                    except:
                        Tracer()()
            
            # check if max depth is reached / further partition is possible
            if len(d.keys()) > len(D[prev_level].keys()) and k<maxLevel:
                D[next_level] = d.copy()
            else:
                print('Max cluster depth: %s'%prev_level)
                print('Time to complete recursive louvain: %s seconds'%str(time.time()-start))
                break
    return D

In [ ]:
D = recursive_louvain(G)

### Export to csv 

In [ ]:
# prepare for csv output
df2 = df[["ocid"]]
for level in D.keys():
    df2[level] = None
    for key,val in D[level].iteritems():
        df2.loc[val,level] = key
df2.head(25)

In [ ]:
df[["ocid","template"]].to_csv("resource_contracts_with_text_louvain.csv")

### reference code --------------------------- 

In [ ]:
from fuzzywuzzy import fuzz
longstr1 = df.iloc[5].contract_text_clean
longstr2 = df.iloc[14].contract_text_clean
fuzz.ratio(longstr1, longstr2)

In [ ]:
minchunk,avgchunk,maxchunk=32,64,128
a = set(rabin_partition(longstr1,avgchunk,minchunk,maxchunk,32))
b = set(rabin_partition(longstr2,avgchunk,minchunk,maxchunk,32))

print
print "EXAMPLE USE CASE: DOCUMENT SIMILARITY"
print

print
print "Percent similarity between document 1 and 2: \n%0.2f%%"%( 100.*len(a&b)/float(min(len(a),len(b))) )

print
print "Common rabin chunks between documents 1 and 2:"
print [k for k in rabin_partition(longstr1) if k in set(rabin_partition(longstr2))]

print
print "Non-common rabin chunks between document 1 and 2:"
print [k for k in rabin_partition(longstr1) if k not in set(rabin_partition(longstr2))]
print

In [ ]:
# clustering based on connected components
def find_cluster_cutoff(G, cutoff = 0.10, minCluster=0):
    print('\nSimilarity cutoff: %f' % cutoff )
    H = G.copy()
    H.remove_edges_from([(u,v) for (u,v,d) in H.edges(data=True) if d['weight'] < cutoff])
    clusters = [sorted(i) for i in sorted(nx.connected_components(H),key=len,reverse=True) if len(i)>minCluster]
    print_cluster_summary(G,clusters)
    return clusters

# summary of cluster results
def print_cluster_summary(G,clusters):
    print('\nNumber of clusters identified: %d' % len(clusters))
    print('Document coverage: %d%% (%d of %d)' % \
          (100 * sum([len(i) for i in clusters])/len(G.node), sum([len(i) for i in clusters]), len(G.node)))
    print('\nCluster sizes:')
    print([len(i) for i in clusters])

# clustering based on lovain method 
def find_cluster_louvain(G,minCluster=0):
    partition = cm.best_partition(G)
    clusters = []
    for label in set(partition.values()):
        clusters.append([i for i in partition.keys() if partition[i] == label])
    clusters = [sorted(i) for i in sorted(clusters, key = len, reverse=True) if len(i)>minCluster]
    print_cluster_summary(G,clusters)
    return clusters

# collect output
def base_output(G,node,label):
    doc_id     = G.node[node]['ocid']
    doc_degree = G.degree(node) 
    return [doc_id,label,doc_degree]